In [1]:
# Add Nicole data to the database

import pandas as pd
import glob
import os


# Path to the folder
folder_path = '/Users/damiano/Desktop/German/Nicole'
file_pattern = "Neues*.xlsx"  # Pattern for files starting with "Neues"


# Path to the Excel file
excel_file = 'database.xlsx'

# Load a new dataframe or the old dataframe
load = "OLD" # "NEW" or "OLD" (THINK BEFORE CLICKING, IT MAY CANCEL THE CURRENT DATABASE!)
if load == "NEW":
    dataframes_dict = {}
elif load == "OLD":
    dataframes_dict = pd.read_excel(excel_file, sheet_name=None)
    # Now dataframes_dict contains all sheets from the Excel file as DataFrames


# Iterate over all files in the folder matching the pattern
for file_path in glob.glob(os.path.join(folder_path, file_pattern)):
    # Extract filename without extension
    file_name = os.path.basename(file_path).split('.')[0]

    # Check if the file name already exists in the dictionary
    if file_name not in dataframes_dict:

        # Load the Excel file
        excel_file = pd.ExcelFile(file_path)

        # Get the number of sheets in the file
        num_sheets = len(excel_file.sheet_names)

        ### 1st sheet ###
        # Load the 1st sheet of the excel file into a DataFrame (sheet 0)
        df_1 = excel_file.parse(sheet_name=0)

        # Rename the 1st and 2nd column
        df_1.columns = ['German', 'English']

        # Clean the DataFrame and create a copy
        cleaned_df_1 = df_1.dropna(how='all').copy()

        # Add weights column
        cleaned_df_1['weights'] = 1

        # Add notes column
        cleaned_df_1['notes'] = ""

        if num_sheets == 1:
            cleaned_df = cleaned_df_1

        elif:
            ### 2nd sheet ###
            # Load the 2nd sheet of the excel file into a DataFrame (sheet 1)
            df_2 = excel_file.parse(sheet_name=1)

            # Assuming the German word is always followed by its English translation and description
            # Create two separate DataFrames for German and English rows
            df_german = df_2.iloc[::2].reset_index(drop=True)  # German words on even rows
            df_english = df_2.iloc[1::2].reset_index(drop=True)  # English translations on odd rows

            # Combine the two DataFrames
            combined_df = pd.concat([df_german, df_english], axis=1)

            # Rename columns
            combined_df.columns = ['German', 'English_Description']

            # Split the English and Description columns
            combined_df[['English', 'notes']] = combined_df['English_Description'].str.split(' - ', expand=True)

            # Now drop the 'English_Description' column
            combined_df.drop(columns=['English_Description'], inplace=True)

            # Clean the DataFrame and create a copy
            cleaned_df_2 = combined_df.dropna(how='all').copy()

            # Add weights column
            cleaned_df_2['weights'] = 1

            # Reorder the columns
            cleaned_df_2 = cleaned_df_2[['German', 'English', 'weights', 'notes']].copy()

            ### Merge the two sheets
            sheets = [cleaned_df_1, cleaned_df_2]
            cleaned_df = pd.concat(sheets)

        elif num_sheets == 3:
            ### 3rd sheet ONLY ###
            # Load the 3rd sheet of the excel file into a DataFrame (sheet 2)
            df_3 = excel_file.parse(sheet_name=2)

            # Assuming that the order of rows is always:
            # 1. the German word 
            # 2. its English translation 
            # 3. its description in German
            # Create 3 separate DataFrames
            df_german = df_3.iloc[0::3].reset_index(drop=True)  # German words, on rows starting with 0 and in steps of 3
            df_english = df_3.iloc[1::3].reset_index(drop=True)  # English translations, on rows starting with 1 and in steps of 3
            df_description = df_3.iloc[2::3].reset_index(drop=True)  # Description, on rows starting with 2 and in steps of 3

            # Combine the two DataFrames
            combined_df = pd.concat([df_german, df_english, df_description], axis=1)

            # Rename columns
            combined_df.columns = ['German', 'English', 'notes']

            # Clean the DataFrame and create a copy
            cleaned_df_2 = combined_df.dropna(how='all').copy()

            # Add weights column
            cleaned_df_2['weights'] = 1

            # Reorder the columns
            cleaned_df_2 = cleaned_df_2[['German', 'English', 'weights', 'notes']].copy()

            ### Merge the two sheets
            sheets = [cleaned_df_1, cleaned_df_2]
            cleaned_df = pd.concat(sheets)
        
        else:
            # Display a message 
            print("\n ### Unrecognized Data Structure", f"The file '{file_name}' \
                                has an unexpected number of sheets.")
 
            
        # reset the index and drop the old index
        cleaned_df.reset_index(drop=True, inplace=True)

        # Store the DataFrame in the dictionary
        dataframes_dict[file_name] = cleaned_df

# Now dataframes_dict contains all the loaded and cleaned DataFrames

dataframes_dict.keys()

dict_keys(['Neues Vokabular--2023-11-17', 'Neues Vokabular--2023-12-07', 'Neues Vokabular--2023-11-20', 'Neues Vokabular--2023-12-01', 'Neues Vokabular--2023-10-30', 'Neues Vokabular--2023-10-26', 'Neues Vokabular--2023-11-27', 'Neues Vokabular--2023-11-30', 'Neues Vokabular--2023-10-27', 'Neues Vokabular--2023-10-31', 'Neues Vokabular--2023-10-28', 'Neues Vokabular--2023-11-25', 'Neues Vokabular--2023-10-25', 'Neues Vokabular--2023-11-24', 'Neues Vokabular--2023-12-02', 'Neues Vokabular--2023-11-29_12h', 'Neues Vokabular--2023-12-04', 'Neues Vokabular--2023-12-08', 'Neues Vokabular--2023-11-29_11h', 'Neues Vokabular--2023-10-23', 'Neues Vokabular--2023-11-22'])

In [5]:
# Save database in excel

with pd.ExcelWriter('database.xlsx') as writer:
    for key, df in dataframes_dict.items():
        df.to_excel(writer, sheet_name=key, index=False)

In [4]:
# Add Preply vocabulary to the database

import pandas as pd
import glob
import os


# Path to the folder
folder_path = '/Users/damiano/Desktop/German/Nicole'
file_pattern = "Vokabular*.xlsx"  # Pattern for files starting with "Neues"

# Path to the Excel file
excel_file = 'database.xlsx'

# Read the database Excel file into a dictionary of DataFrames
dataframes_dict = pd.read_excel(excel_file, sheet_name=None)

# Path to the folder
file_path = glob.glob(os.path.join(folder_path, file_pattern))[0]
print(file_path)
# Extract filename without extension
file_name = os.path.basename(file_path).split('.')[0]
df = pd.read_excel(file_path)

# Assuming the German word is always followed by its English translation and description
# Create two separate DataFrames for German and English rows
df_german = df.iloc[::2].reset_index(drop=True)  # German words on even rows
df_english = df.iloc[1::2].reset_index(drop=True)  # English translations on odd rows

# Combine the two DataFrames
combined_df = pd.concat([df_german, df_english], axis=1)

# Rename columns
combined_df.columns = ['German', 'English_Description']

# Split the English and Description columns
combined_df[['English', 'notes']] = combined_df['English_Description'].str.split(' - ', expand=True)

# Now drop the 'English_Description' column
combined_df.drop(columns=['English_Description'], inplace=True)

# Clean the DataFrame and create a copy
cleaned_df = combined_df.dropna(how='all').copy()

# Add weights column
cleaned_df['weights'] = 1

# Reorder the columns
cleaned_df = cleaned_df[['German', 'English', 'weights', 'notes']].copy()

# reset the index and drop the old index
cleaned_df.reset_index(drop=True, inplace=True)

for i in range(0, len(cleaned_df), 25):
    df_cut = cleaned_df.iloc[i:i + 25]
    df_cut.reset_index(drop=True, inplace=True) 
    dataframes_dict[f'Vok Preply_{i // 25 + 1}'] = df_cut

dataframes_dict.keys()

/Users/damiano/Desktop/German/Nicole/Vokabular Preply -- bis 2023-12-07.xlsx


dict_keys(['Neues Vokabular--2023-11-17', 'Neues Vokabular--2023-12-07', 'Neues Vokabular--2023-11-20', 'Neues Vokabular--2023-12-01', 'Neues Vokabular--2023-10-30', 'Neues Vokabular--2023-10-26', 'Neues Vokabular--2023-11-27', 'Neues Vokabular--2023-11-30', 'Neues Vokabular--2023-10-27', 'Neues Vokabular--2023-10-31', 'Neues Vokabular--2023-10-28', 'Neues Vokabular--2023-11-25', 'Neues Vokabular--2023-10-25', 'Neues Vokabular--2023-11-24', 'Neues Vokabular--2023-12-02', 'Neues Vokabular--2023-11-29_12h', 'Neues Vokabular--2023-12-04', 'Neues Vokabular--2023-12-08', 'Neues Vokabular--2023-11-29_11h', 'Neues Vokabular--2023-10-23', 'Neues Vokabular--2023-11-22', 'Vok Preply_1', 'Vok Preply_2', 'Vok Preply_3', 'Vok Preply_4', 'Vok Preply_5', 'Vok Preply_6', 'Vok Preply_7', 'Vok Preply_8', 'Vok Preply_9', 'Vok Preply_10', 'Vok Preply_11', 'Vok Preply_12', 'Vok Preply_13', 'Vok Preply_14'])